In [43]:
# Cell 1 – Cài thư viện (chạy ~2-3 phút)
!pip install -q --no-cache-dir torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --no-cache-dir speechbrain==1.0.3


In [44]:
# Cell 2 – Khai báo đường dẫn & import (cho private-test)
from pathlib import Path
import torch, torchaudio, torch.nn.functional as F
import tqdm, functools, zipfile, os

# 1) Đường dẫn đến checkpoints
CKPT_ROOT    = Path("/kaggle/input/checkpointresult/checkpoints")

# 2) Thư mục gốc private-test
PRIVATE_ROOT = Path("/kaggle/input/private-test")

# 3) Lấy folder timestamp con (chứa audio)
subfolders = [p for p in PRIVATE_ROOT.iterdir() if p.is_dir()]
assert subfolders, f"Không tìm thấy thư mục con trong {PRIVATE_ROOT}"
sub = subfolders[0]

# 4) Folder chứa .wav
WAV_DIR   = sub / "private-test-data-sv"

# 5) File CSV nằm ngay trong PRIVATE_ROOT (không phải trong `sub`)
PAIR_FILE = PRIVATE_ROOT / "prompts_sv.csv"

# 6) Đường dẫn đầu ra
OUT_TXT   = Path("/kaggle/working/predictions.txt")
ZIP_PATH  = Path("/kaggle/working/submission.zip")

# 7) Kiểm tra tồn tại
print("✅ WAV_DIR exists?   ", WAV_DIR.exists(),   "→", WAV_DIR)
print("✅ PAIR_FILE exists?", PAIR_FILE.exists(),  "→", PAIR_FILE)


✅ WAV_DIR exists?    True → /kaggle/input/private-test/private-test-data-sv-20250512T023803Z-001/private-test-data-sv
✅ PAIR_FILE exists? True → /kaggle/input/private-test/prompts_sv.csv


In [45]:
# --- CELL 3 -------------------------------------------------------
from speechbrain.inference.speaker import EncoderClassifier
from pathlib import Path
import torch, yaml, re, pprint

# 1) Nạp ECAPA gốc (SpeechBrain)
pretrained = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_ecapa",
    run_opts={"device": "cuda"}
)
compute_features = pretrained.mods.compute_features
mean_var_norm   = pretrained.mods.mean_var_norm
embedding_model = pretrained.mods.embedding_model
classifier      = pretrained.mods.classifier

# 2) Tìm thư mục checkpoint mới nhất
ckpt_dirs = sorted([d for d in CKPT_ROOT.iterdir() if d.is_dir()],
                   key=lambda p: p.name)
assert ckpt_dirs, "❌  Không tìm thấy folder CKPT+…"
latest = ckpt_dirs[-1]
print("🔑  Dùng checkpoint:", latest.name)

# 3) Đọc file CKPT.yaml để biết cấu trúc
ckpt_yaml = latest / "CKPT.yaml"
assert ckpt_yaml.exists(), f"❌  Không thấy {ckpt_yaml}"
with open(ckpt_yaml) as f:
    ckpt_meta = yaml.safe_load(f)

# ckpt_meta["modules"] là map recoverable → file; thường mọi thứ ở 'brain.ckpt'
param_map = ckpt_meta.get("modules", {})
print("📑  modules map:", param_map)

# 4) Nạp brain.ckpt với pickle (weights_only=False – mặc định)
brain_ckpt = latest / param_map.get("brain", "optimizer.ckpt")
assert brain_ckpt.exists(), f"❌  Không thấy {brain_ckpt}"
state = torch.load(brain_ckpt, map_location="cpu")   # ← pickle đầy đủ
print("📂  Keys trong brain.ckpt:", list(state.keys())[:8], "...")

# 5) Lấy state_dict cho embedding_model và classifier
# Trường hợp A: brain.ckpt chứa dict 2 cấp (embedding_model -> dict)
if isinstance(state.get("embedding_model"), dict):
    emb_state = state["embedding_model"]
    clf_state = state["classifier"]
# Trường hợp B: phẳng → lọc prefix
else:
    def slice_state(prefix):
        return {k[len(prefix)+1:]: v for k, v in state.items()
                if k.startswith(prefix + ".")}
    emb_state = slice_state("embedding_model")
    clf_state = slice_state("classifier")

embedding_model.load_state_dict(emb_state, strict=False)
classifier.load_state_dict(clf_state,   strict=False)

# (tùy) nếu có mean_var_norm trong brain.ckpt
if "mean_var_norm" in state:
    mean_var_norm.load_state_dict(state["mean_var_norm"], strict=False)

# 6) Chuyển mọi mô-đun sang GPU
for mod in [compute_features, mean_var_norm,
            embedding_model, classifier]:
    mod.to("cuda")

print("✅  Nạp checkpoint thành công – mô hình sẵn sàng inference")


/usr/local/lib/python3.11/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

🔑  Dùng checkpoint: CKPT+2025-05-16+23-44-53+00
📑  modules map: {}
📂  Keys trong brain.ckpt: ['state', 'param_groups'] ...
✅  Nạp checkpoint thành công – mô hình sẵn sàng inference


In [46]:
# Cell 4 – Hàm lấy embedding + cosine score (có cache)

@functools.lru_cache(maxsize=None)
@torch.inference_mode()
def get_embed(wav_path: str):
    sig, _ = torchaudio.load(wav_path)
    if sig.dim() == 2:                      # stereo → mono
        sig = sig.mean(0, keepdim=True)

    feats = mean_var_norm(
        compute_features(sig.to("cuda")),
        torch.tensor([sig.shape[-1]], device="cuda")
    )
    emb = embedding_model(feats, torch.tensor([feats.shape[1]], device="cuda"))
    if emb.dim() == 3:                      # [1,T,192] → mean T
        emb = emb.mean(1)
    return F.normalize(emb.squeeze(0), p=2, dim=-1).half()   # (192,)

def cosine_score(e1, e2):
    return float((e1 * e2).sum())


In [47]:
# Cell 5 – Sinh predictions.txt + submission.zip (private-test)
import pandas as pd

# Đọc file prompts_sv.csv: mỗi dòng có hai tên file ngăn cách bởi khoảng trắng
df = pd.read_csv(
    PAIR_FILE,
    delim_whitespace=True,    # << quan trọng: parse whitespace
    header=None,
    names=["wav1", "wav2"]
)

# Tạo file predictions.txt
with open(OUT_TXT, "w") as fout:
    for _, row in tqdm.tqdm(df.iterrows(), total=len(df), desc="Scoring pairs"):
        w1, w2 = row.wav1, row.wav2
        p1 = (WAV_DIR / w1).as_posix()
        p2 = (WAV_DIR / w2).as_posix()
        emb1 = get_embed(p1)
        emb2 = get_embed(p2)
        fout.write(f"{cosine_score(emb1, emb2):.6f}\n")

print(f"✨  Đã ghi {len(df)} scores vào {OUT_TXT.name}")

# Nén zip để nộp
with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(OUT_TXT, arcname="predictions.txt")
print("🎉  File nộp sẵn sàng →", ZIP_PATH)


/tmp/ipykernel_35/4052694161.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
Scoring pairs: 100%|██████████| 15971/15971 [03:37<00:00, 73.44it/s] 

✨  Đã ghi 15971 scores vào predictions.txt
🎉  File nộp sẵn sàng → /kaggle/working/submission.zip


In [50]:
# Liệt kê nội dung /kaggle/working để xem có submission.zip không
!ls -lh /kaggle/working


total 200K
-rw-r--r-- 1 root root 140K May 30 09:47 predictions.txt
drwxr-xr-x 2 root root 4.0K May 30 09:24 pretrained_ecapa
-rw-r--r-- 1 root root  49K May 30 09:47 submission.zip


In [48]:
# Cell 6 (tuỳ chọn) – Tính EER offline nếu có nhãn

import pandas as pd, numpy as np
from sklearn.metrics import roc_curve

GT_CSV = VOX_ROOT / "test_list_gt.csv"
if GT_CSV.exists():
    gt = pd.read_csv(GT_CSV)          # cột cuối = label (0/1)
    scores = np.loadtxt(OUT_TXT)
    labels = gt.iloc[:, -1].to_numpy()

    fpr, tpr, _ = roc_curve(labels, scores, pos_label=1)
    eer = fpr[np.nanargmin(np.abs(1 - tpr - fpr))] * 100
    print(f"🔎  EER offline = {eer:.2f} %")
else:
    print("⚠️  Không tìm thấy test_list_gt.csv – bỏ qua bước tính EER.")


NameError: name 'VOX_ROOT' is not defined